In [ ]:
import pathlib
import sys
import warnings

import autograd
import autograd.numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import autocrit

In [ ]:
sys.path.append("..")

import shared.format
import shared.tools

In [ ]:
plt.rcParams["font.size"] = shared.format.FONTSIZE

In [ ]:
pd.options.mode.chained_assignment = None

warnings.filterwarnings(
    "ignore", category=FutureWarning,
    message="Using a non-tuple sequence for multidimensional indexing is deprecated;",
    module="autograd")

In [ ]:
import autocrit

In [ ]:
import autocrit_tools
import autocrit_tools.linearpaper.utils as utils

In [ ]:
pickle_dir = pathlib.Path("data") / "linear-results"

In [ ]:
analytical_cp_df = pd.read_pickle(pickle_dir / "analytical_cp_df.pkl")

def compute_index(spectrum, eigval_cutoff=1e-5):
    return np.mean(spectrum < -eigval_cutoff)

analytical_cp_df["morse_index"] = analytical_cp_df["hessian_spectrum"].apply(compute_index)

In [ ]:
cutoffs = [1e-10, 1e-6, 1e-4, np.inf]
cutoff_strings = [r"$\varepsilon=$1e-10", r"$\varepsilon=$1e-6", r"$\varepsilon=$1e-4", "no cutoff"]

In [ ]:
cutoff_fnames = ["1e-10", "1e-6", "1e-4", "none"]

cutoff_cp_df_names= [pickle_dir / ("cutoff_" + cutoff_fname + ".pkl")
                     for cutoff_fname in cutoff_fnames]

reloaded_cutoff_cp_dfs = [pd.read_pickle(cutoff_cp_df_name)
                          for cutoff_cp_df_name in cutoff_cp_df_names]

In [ ]:
import string

uppercase_letters = string.ascii_uppercase

In [ ]:
labelsize = "x-large"
titlesize = "xx-large"

fig, axs = plt.subplots(figsize=(12, 12), nrows=2, ncols=2)

[ax.set_xlabel("Index", size=labelsize)
  for ax in axs[1, :]]

[ax.set_ylabel(r"$L(\theta)$", size=labelsize)
  for ax in axs[:, 0]]

[ax.scatter(
    analytical_cp_df["morse_index"], analytical_cp_df["cost"], color="gray",
    label="analytical")
 for ax in axs.flatten()]

[ax.scatter(
    cutoff_cp_df["morse_index"], cutoff_cp_df["candidate_loss"], color="C0",
    label="numerical")
 for cutoff_cp_df, ax in zip(reloaded_cutoff_cp_dfs, axs.flatten())]

[ax.text(0.05, 0.9, cutoff_string, size=titlesize, transform=ax.transAxes)
 for cutoff_string, ax in zip(cutoff_strings, axs.flatten())]

axs[-1, -1].legend(loc="lower right");

[shared.tools.add_panel_label(letter, ax, pos=(-0.1, 1.06))
 for letter, ax in zip(uppercase_letters, axs.flatten())]

plt.tight_layout();
plt.savefig("cutoffs.pdf")